**IMPORTS AND DATA PREPROCESSING**

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2024-03-06 17:22:06.462139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 17:22:06.462186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 17:22:06.463891: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 17:22:06.473521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 17:22:07.653790: W tensorflow/compiler/tf2

In [2]:
csv_column_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
species = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file('iris_train.csv',"https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file('iris_test.csv',"https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path,names=csv_column_names,header=0)
test = pd.read_csv(test_path,names=csv_column_names,header=0)

In [4]:
train

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0
...,...,...,...,...,...
115,5.5,2.6,4.4,1.2,1
116,5.7,3.0,4.2,1.2,1
117,4.4,2.9,1.4,0.2,0
118,4.8,3.0,1.4,0.1,0


In [5]:
test.shape

(30, 5)

In [6]:
X_train = train[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]
X_test = test[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]

y_train = train[['Species']]
y_test = test[['Species']]

In [7]:
X_train

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3
...,...,...,...,...
115,5.5,2.6,4.4,1.2
116,5.7,3.0,4.2,1.2
117,4.4,2.9,1.4,0.2
118,4.8,3.0,1.4,0.1


In [8]:
y_train

,Species
0,2
1,1
2,2
3,0
4,0
...,...
115,1
116,1
117,0
118,0


**DEFINING THE INPUT FUNCTION AND FEATURE COLUMNS**

In [9]:
def input_fn(features,labels,training=True,batch_size=1000):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [10]:
train_input_fn = input_fn(X_train,y_train)
test_input_fn = input_fn(X_test,y_test)

2024-03-06 17:22:09.736271: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
feature_cols = []

for key in X_train.keys():
    feature_cols.append(tf.feature_column.numeric_column(key=key))

feature_cols

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

**BUILDING AND TRAINING THE MODEL**

In [12]:
model = tf.estimator.DNNClassifier(feature_columns=feature_cols,hidden_units=[30,10],n_classes=3)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdpqr6b3i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster'

In [16]:
model.train(input_fn= lambda : input_fn(X_train,y_train,training=True),steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdpqr6b3i/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 30000...
INFO:tensorflow:Saving checkpoints for 30000 into /tmp/tmpdpqr6b3i/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 30000...
INFO:tensorflow:loss = 0.119227536, step = 30000
INFO:tensorflow:global_step/sec: 172.287
INFO:tensorflow:loss = 0.11953955, step = 30100 (0.582 sec)
INFO:tensorflow:global_step/sec: 185.615
INFO:tensorflow:loss = 0.116488814, step = 30200 (0.539 sec)
INFO:tensorflow:global_step/sec: 166.298
INFO:tensorflow:loss = 0.117080465, step = 30300 (0.602 sec)
INFO:tensorflow:global_step/sec: 188.07
INFO:tensorflow:loss = 0.120307155, step = 30400 (0.531 s

**EVALUATING THE MODEL**

In [23]:
result = model.evaluate(input_fn=lambda: input_fn(X_test, y_test, training=False))
print('Accuracy: ', result['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-03-06T17:32:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdpqr6b3i/model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28237s
INFO:tensorflow:Finished evaluation at 2024-03-06-17:32:38
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.96666664, average_loss = 0.082608595, global_step = 50000, loss = 0.082608595
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 50000: /tmp/tmpdpqr6b3i/model.ckpt-50000
Accuracy:  0.96666664


**As we can see our model shows a 96.67% accuracy on the testing data, which is pretty good.**